In [26]:
import pandas as pd
import os

# Check out this nifty thing. I pass the data frame through this later to format the whole thing
def formatData(data):
    try:
        return f"{data:,.2f}"
    except:
        ValueError
        return(data)

# Get the resource files
schoolDataPath = os.path.join("Resources", "schools_complete.csv")
studentDataPath = os.path.join("Resources", "students_complete.csv")
schoolData = pd.read_csv(schoolDataPath)
studentData = pd.read_csv(studentDataPath)

# Collect and mathatize all the data for the district summary dataframe
schoolNames = schoolData['school_name'].unique()
numStudents = schoolData['size'].sum()
districtPassingMath = (studentData['math_score'] >= 70).sum() / numStudents * 100
districtPassingReading = (studentData['reading_score'] >= 70).sum() / numStudents * 100
districtOverallPassing = ((studentData['math_score'] >= 70) & (studentData['reading_score'] >= 70)).sum() / numStudents * 100

# Create the dataframe for the district summary, and format the data.
districtSummary = pd.DataFrame({
    "District Wide": [len(schoolNames),                                 # Number of schools
    '{:,}'.format(numStudents),                                         # Number of students
    '${:,}'.format(schoolData['budget'].sum()),                         # Total budget
    '{:.2f}'.format(studentData['math_score'].mean()),                  # Average math score
    '{:.2f}'.format(studentData['reading_score'].mean()),               # Average reading score
    '{:.2f}%'.format(districtPassingMath),                              # Percent passing math
    '{:.2f}%'.format(districtPassingReading),                           # Percent passing reading
    '{:.2f}%'.format(districtOverallPassing)                            # Percent overall passing
    ]},index = pd.Index(["Number of Schools", "Total Students", "Total Budget", "Average Math Score",
        "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]))

# Collect and mathatize the data for the school summary dataframe
groupedSchoolData = studentData.groupby("school_name", sort = False)
averagePassingMath = groupedSchoolData['math_score'].mean().values
averagePassingReading = groupedSchoolData['reading_score'].mean().values
passingMath = (groupedSchoolData['math_score'].apply(lambda x: (x >= 70).sum())) / schoolData['size'].values * 100
passingReading = (groupedSchoolData['reading_score'].apply(lambda x: (x >= 70).sum())) / schoolData['size'].values * 100
studentData["pass both"] = (studentData["math_score"] >= 70) & (studentData["reading_score"] >= 70)
overallPassing = groupedSchoolData["pass both"].mean() * 100
spendingBins = [0, 585, 630, 645, 680]
spendingLabels = ["<$585", "$585-630", "$630-645", "$645-680"]
sizeBins = [0, 1000, 2000, 5000]
sizeLabels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Create a dataframe for the school summary and format the data.
schoolSummary = pd.DataFrame({
    "School Name": schoolNames,
    "School Type": schoolData["type"],
    "Total Students": schoolData['size'],
    "Total School Budget": schoolData['budget'],
    "Per Student Budget": (schoolData['budget'] / schoolData['size']),
    "Average Math Score": averagePassingMath,
    "Average Reading Score": averagePassingReading,
    "% Passing Math": passingMath.values,
    "% Passing Reading": passingReading.values,
    "% Overall Passing": overallPassing.values,
    "Spending Ranges (Per Student)": pd.cut(schoolData['budget'] / schoolData['size'],
                                            bins = spendingBins, labels =spendingLabels),
    "School Size":pd.cut(schoolData['size'], bins = sizeBins, labels= sizeLabels) 
    })

# Create a dataframe for the summary of grades based on school spending
schoolGroupedSpending = schoolSummary.groupby("Spending Ranges (Per Student)", sort = False)
spendingSummary = pd.DataFrame({
    "Average Math Score": schoolGroupedSpending['Average Math Score'].mean(),
    "Average Reading Score": schoolGroupedSpending['Average Reading Score'].mean(),
    "% Passing Math": schoolGroupedSpending['% Passing Math'].mean(),
    "% Passing Reading": schoolGroupedSpending['% Passing Reading'].mean(),
    "% Overall Passing": schoolGroupedSpending['% Overall Passing'].mean()
})
spendingSummary = spendingSummary.applymap(formatData)

#  Create a dataframe for the summary of grades based on school size
schoolGroupedSize = schoolSummary.groupby("School Size", sort = False)
sizeSummary = pd.DataFrame({
    "Average Math Score": schoolGroupedSize['Average Math Score'].mean(),
    "Average Reading Score": schoolGroupedSize['Average Reading Score'].mean(),
    "% Passing Math": schoolGroupedSize['% Passing Math'].mean(),
    "% Passing Reading": schoolGroupedSize['% Passing Reading'].mean(),
    "% Overall Passing": schoolGroupedSize['% Overall Passing'].mean()
})
sizeSummary = sizeSummary.applymap(formatData)

# Create a dataframe for the summary of grades based on school type
schoolGroupedType = schoolSummary.groupby("School Type", sort = False)
typeSummary = pd.DataFrame({
    "Average Math Score": schoolGroupedType['Average Math Score'].mean(),
    "Average Reading Score": schoolGroupedType['Average Reading Score'].mean(),
    "% Passing Math": schoolGroupedType['% Passing Math'].mean(),
    "% Passing Reading": schoolGroupedType['% Passing Reading'].mean(),
    "% Overall Passing": schoolGroupedType['% Overall Passing'].mean()
})
typeSummary = typeSummary.applymap(formatData)
schoolSummary = schoolSummary.applymap(formatData)

# Sort the school summary by school name, then highest and lowest performing.
schoolSummary.sort_values('School Name', inplace = True)
highestPerformingSchools = schoolSummary.sort_values('% Overall Passing', ascending = False)
lowestPerformingSchools = schoolSummary.sort_values('% Overall Passing')

# Make a pivot table for math scores by grade and format it to look all nice and stuff
mathScoresByGrade = pd.pivot_table(studentData, index='school_name', columns='grade',
                                   values='math_score', aggfunc='mean')
mathScoresByGrade = mathScoresByGrade.reindex(columns = ['9th', '10th', '11th', '12th'])

# Do all the same stuff here but for reading
readingScoresByGrade = pd.pivot_table(studentData, index='school_name', columns='grade',
                                      values='reading_score', aggfunc='mean')
readingScoresByGrade = readingScoresByGrade.reindex(columns = ['9th', '10th', '11th', '12th'])

In [4]:
districtSummary

,District Wide
Number of Schools,15
Total Students,"39,170"
Total Budget,"$24,649,428"
Average Math Score,78.99
Average Reading Score,81.88
% Passing Math,74.98%
% Passing Reading,85.81%
% Overall Passing,65.17%


In [5]:
schoolSummary

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student),School Size
7,Bailey High School,District,"4,976.00","3,124,928.00",628.00,77.05,81.03,66.68,81.93,54.64,$585-630,Large (2000-5000)
6,Cabrera High School,Charter,"1,858.00","1,081,356.00",582.00,83.06,83.98,94.13,97.04,91.33,<$585,Medium (1000-2000)
1,Figueroa High School,District,"2,949.00","1,884,411.00",639.00,76.71,81.16,65.99,80.74,53.20,$630-645,Large (2000-5000)
13,Ford High School,District,"2,739.00","1,763,916.00",644.00,77.10,80.75,68.31,79.30,54.29,$630-645,Large (2000-5000)
4,Griffin High School,Charter,"1,468.00","917,500.00",625.00,83.35,83.82,93.39,97.14,90.60,$585-630,Medium (1000-2000)
3,Hernandez High School,District,"4,635.00","3,022,020.00",652.00,77.29,80.93,66.75,80.86,53.53,$645-680,Large (2000-5000)
8,Holden High School,Charter,427.00,"248,087.00",581.00,83.80,83.81,92.51,96.25,89.23,<$585,Small (<1000)
0,Huang High School,District,"2,917.00","1,910,635.00",655.00,76.63,81.18,65.68,81.32,53.51,$645-680,Large (2000-5000)
12,Johnson High School,District,"4,761.00","3,094,650.00",650.00,77.07,80.97,66.06,81.22,53.54,$645-680,Large (2000-5000)
9,Pena High School,Charter,962.00,"585,858.00",609.00,83.84,84.04,94.59,95.95,90.54,$585-630,Small (<1000)


In [6]:
highestPerformingSchools

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student),School Size
6,Cabrera High School,Charter,"1,858.00","1,081,356.00",582.00,83.06,83.98,94.13,97.04,91.33,<$585,Medium (1000-2000)
14,Thomas High School,Charter,"1,635.00","1,043,130.00",638.00,83.42,83.85,93.27,97.31,90.95,$630-645,Medium (1000-2000)
4,Griffin High School,Charter,"1,468.00","917,500.00",625.00,83.35,83.82,93.39,97.14,90.60,$585-630,Medium (1000-2000)
5,Wilson High School,Charter,"2,283.00","1,319,574.00",578.00,83.27,83.99,93.87,96.54,90.58,<$585,Large (2000-5000)
9,Pena High School,Charter,962.00,"585,858.00",609.00,83.84,84.04,94.59,95.95,90.54,$585-630,Small (<1000)
10,Wright High School,Charter,"1,800.00","1,049,400.00",583.00,83.68,83.95,93.33,96.61,90.33,<$585,Medium (1000-2000)
2,Shelton High School,Charter,"1,761.00","1,056,600.00",600.00,83.36,83.73,93.87,95.85,89.89,$585-630,Medium (1000-2000)
8,Holden High School,Charter,427.00,"248,087.00",581.00,83.80,83.81,92.51,96.25,89.23,<$585,Small (<1000)
7,Bailey High School,District,"4,976.00","3,124,928.00",628.00,77.05,81.03,66.68,81.93,54.64,$585-630,Large (2000-5000)
13,Ford High School,District,"2,739.00","1,763,916.00",644.00,77.10,80.75,68.31,79.30,54.29,$630-645,Large (2000-5000)


In [7]:
lowestPerformingSchools

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student),School Size
11,Rodriguez High School,District,"3,999.00","2,547,363.00",637.00,76.84,80.74,66.37,80.22,52.99,$630-645,Large (2000-5000)
1,Figueroa High School,District,"2,949.00","1,884,411.00",639.00,76.71,81.16,65.99,80.74,53.20,$630-645,Large (2000-5000)
0,Huang High School,District,"2,917.00","1,910,635.00",655.00,76.63,81.18,65.68,81.32,53.51,$645-680,Large (2000-5000)
3,Hernandez High School,District,"4,635.00","3,022,020.00",652.00,77.29,80.93,66.75,80.86,53.53,$645-680,Large (2000-5000)
12,Johnson High School,District,"4,761.00","3,094,650.00",650.00,77.07,80.97,66.06,81.22,53.54,$645-680,Large (2000-5000)
13,Ford High School,District,"2,739.00","1,763,916.00",644.00,77.10,80.75,68.31,79.30,54.29,$630-645,Large (2000-5000)
7,Bailey High School,District,"4,976.00","3,124,928.00",628.00,77.05,81.03,66.68,81.93,54.64,$585-630,Large (2000-5000)
8,Holden High School,Charter,427.00,"248,087.00",581.00,83.80,83.81,92.51,96.25,89.23,<$585,Small (<1000)
2,Shelton High School,Charter,"1,761.00","1,056,600.00",600.00,83.36,83.73,93.87,95.85,89.89,$585-630,Medium (1000-2000)
10,Wright High School,Charter,"1,800.00","1,049,400.00",583.00,83.68,83.95,93.33,96.61,90.33,<$585,Medium (1000-2000)


In [8]:
spendingSummary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
$645-680,77.00,81.03,66.16,81.13,53.53
$630-645,78.52,81.62,73.48,84.39,62.86
$585-630,81.90,83.16,87.13,92.72,81.42
<$585,83.46,83.93,93.46,96.61,90.37


In [9]:
sizeSummary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Large (2000-5000),77.75,81.34,69.96,82.77,58.29
Medium (1000-2000),83.37,83.86,93.60,96.79,90.62
Small (<1000),83.82,83.93,93.55,96.10,89.88


In [10]:
typeSummary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
District,76.96,80.97,66.55,80.80,53.67
Charter,83.47,83.90,93.62,96.59,90.43


In [24]:
mathScoresByGrade

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [27]:
readingScoresByGrade

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
